In [20]:
# Importing necessary libraries (see configuration file climaticpark_env.yml in github repo)
#!pip install pybdshadow contextily folium pillow timezonefinder plotly

In [36]:
# Library Imports
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import datetime
from datetime import datetime as dt 
from datetime import timedelta
import math

import branca

#from statsmodels.tsa.api import VAR
#from statsmodels.tsa.stattools import adfuller, grangercausalitytests
#from statsmodels.tools.eval_measures import rmse, aic
#from statsmodels.tsa.vector_ar.vecm import coint_johansen
#from statsmodels.stats.stattools import durbin_watson
#import folium
import geopandas as gpd
from shapely.geometry import Polygon
#import pytz
#import pybdshadow
#from timezonefinder import TimezoneFinder
#import json
#import seaborn as sns
#from tabulate import tabulate
#import plotly.express as px
#from sklearn.mixture import GaussianMixture
#import pickle
import os

from suncalc import get_position
from pyproj import CRS,Transformer

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D,MaxPooling1D
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping

import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'branca'

In [ ]:
class ShadowModule:
        
    def __init__(self,lat, lon, roofs, spaces):
        self.lat= lat
        self.lon= lon
        self.roofs = roofs
        self.spaces= spaces
    
    
    def compute_coverage_rates(self, days_lst):
        """
        Compute coverage rates for each day in days_lst
        """

        # DataFrames to store results
        #coverage_rates_df = pd.DataFrame()

        #print(self.spaces.head())
        #print(self.spaces.info())
        #print(self.spaces.geometry)
        #print(self.spaces['geometry'].apply(type).unique())  # Asegúrate de que todas sean geometrías válidas

        coverage_rates_gdf = gpd.GeoDataFrame(geometry=self.spaces.geometry)
        coverage_rates_gdf = coverage_rates_gdf.set_crs(epsg=4326)
        #coverage_rates_gdf = coverage_rates_gdf.to_crs(epsg=self.spaces.crs.to_epsg())

        for current_date in days_lst:
            # Compute shadow projections for roofs

            for hour in range(6,22):
                
                date_hour = dt.combine(current_date, datetime.time(hour, 0)) 

                #try:
                shadows_gdf = self._all_sunshadeshadow_sunlight(date_hour)

                # Calculate coverage rates
                intersection = gpd.overlay(coverage_rates_gdf, shadows_gdf, how='intersection')

                """
                print("*"*8,"shadows_gdf","*"*8)
                print(shadows_gdf.info())
                print(shadows_gdf.head(2))                
                print("*"*8,"intersection","*"*8)
                print(intersection.info())
                print(intersection.head(2))
                print("*"*8,"coverage_rates_gdf","*"*8,)
                print(coverage_rates_gdf.info())
                print(coverage_rates_gdf.head(2))                
                print("*"*8)
                """

                coverage_rates= []
                for index, parking_space in coverage_rates_gdf.iterrows():
                    #print(parking_space)
                    space_total_area = parking_space.geometry.area
                    space_shadow_area = intersection.loc[index, "geometry"].area if index in intersection.index else 0 #intersection.loc[index,"geometry"].area
                    space_coverage= space_shadow_area / space_total_area
                    #print(index, space_total_area,space_shadow_area,space_coverage)
                    #print("*"*8)
                    coverage_rates.append(space_coverage)

                #intersection_area = intersection.geometry.area.sum()
                #parking_space_area = coverage_rates_gdf.geometry.area.sum()

                #coverage_rates = intersection_area / parking_space_area
                #print(coverage_rates)

                """
                coverage_rates = []
                for index, parking_space in self.spaces.iterrows():
                    parking_space_gdf = gpd.GeoDataFrame(geometry=self..geometry)
                    parking_space_gdf = parking_space_gdf.set_crs(epsg=4326)
                    parking_space_gdf = parking_space_gdf.to_crs(epsg=shadows_gdf.crs.to_epsg())

                    intersection = gpd.overlay(parking_space_gdf, shadows_gdf, how='intersection')

                    intersection_area = intersection.geometry.area.sum()
                    parking_space_area = parking_space_gdf.geometry.area.sum()

                    coverage_rate = intersection_area / parking_space_area
                    coverage_rates.append(coverage_rate)
                """

                coverage_rates_gdf[f'coverage_rate_{date_hour.strftime("%Y-%m-%d %H:%M")}'] = coverage_rates
                #except Exception as e:
                #    print(f"ERROR:: ShadowModule compute_coverage_rates: {e}")
        self._coverage_rates = coverage_rates_gdf
        return coverage_rates_gdf

        # Define function to calculate shadow and sunlight for all rooftops
    def _all_sunshadeshadow_sunlight(self, date):
        roof_projected_df= self.roofs.copy()
        roof_projected_df['geometry'] = roof_projected_df['geometry'].apply(lambda r: self._sunshadeshadow_sunlight(np.datetime64(date), r))
        return roof_projected_df


    def _sunshadeshadow_sunlight(self, date, r, sunshade_height=2):
        meanlon= r.centroid.y
        meanlat= r.centroid.x
        # obtain sun position
        sunPosition = get_position(date, meanlon, meanlat)
        if sunPosition['altitude'] < 0:
            raise ValueError("Given time before sunrise or after sunset")
            
        r_coords= np.array(r.exterior.coords)
        r_coords= r_coords.reshape(1,-1,2)
        shape = ShadowModule.lonlat2aeqd(r_coords,meanlon,meanlat)
        azimuth = sunPosition['azimuth']
        altitude = sunPosition['altitude']

        n = np.shape(shape)[0]
        distance = sunshade_height / math.tan(altitude)

        # calculate the offset of the projection position
        lonDistance = distance * math.sin(azimuth)
        latDistance = distance * math.cos(azimuth)

        shadowShape = np.zeros((1, 5, 2))
        shadowShape[:, :, :] += shape
        shadowShape[:, :, 0] = shape[:, :, 0] + lonDistance
        shadowShape[:, :, 1] = shape[:, :, 1] + latDistance
        shadowShape = ShadowModule.aeqd2lonlat(shadowShape,meanlon,meanlat)
        p = Polygon([[p[0], p[1]] for p in shadowShape[0]])
        return p
    
    @staticmethod
    def lonlat2aeqd(lonlat, center_lon, center_lat):
        epsg = CRS.from_proj4("+proj=aeqd +lat_0="+str(center_lat) +
                            " +lon_0="+str(center_lon)+" +datum=WGS84")
        transformer = Transformer.from_crs("EPSG:4326", epsg, always_xy=True)
        proj_coords = transformer.transform(lonlat[:, :, 0], lonlat[:, :, 1])
        proj_coords = np.array(proj_coords).transpose([1, 2, 0])
        return proj_coords
    
    @staticmethod
    def aeqd2lonlat(proj_coords,meanlon,meanlat):
        epsg = CRS.from_proj4("+proj=aeqd +lat_0="+str(meanlat)+" +lon_0="+str(meanlon)+" +datum=WGS84")
        transformer = Transformer.from_crs( epsg,"EPSG:4326",always_xy = True)
        lonlat = transformer.transform(proj_coords[:,:,0], proj_coords[:,:,1])
        lonlat = np.array(lonlat).transpose([1,2,0])
        return lonlat



In [23]:
class DemandModule:
    def __init__(self, entry_exit_tuples):        
        self._entry_exit_tuples= entry_exit_tuples

    def train_demand_predictors(self, lr=0.9, sequence_length= 12, show_details=True, refresh_model=False):
        self._sequence_length= sequence_length

        #self._gm = GaussianMixture(n_components=self._n_components, random_state=42)
        #self._gm.fit(self._entry_exit_tuples[['enter_hour', 'exit_hour']])

        self._gmm_models={}
        for hour, group in self._entry_exit_tuples.groupby('enter_hour'):
            X = group["exit_hour"].values  # Extraer los datos de las características para este grupo
            if X.shape[0]== 1:
                X= X.reshape(1, -1)
            else:
                X= X.reshape(-1,1)
            if X.shape[0]>=2:
                # Crear y ajustar el modelo GMM; ajusta n_components según tus necesidades
                gmm = GaussianMixture(n_components=1, random_state=42)
                gmm.fit(X)
                self._gmm_models[hour] = gmm

        model_path = os.path.join('_models', 'demand_cnnlstm_model.keras')
        # Cargamos o entrenamos el modelo

        n_incoming_veh_df= self._entry_exit_tuples.groupby('date').size().reset_index()
        n_incoming_veh_df['datetime'] = pd.to_datetime(n_incoming_veh_df['date'])

        # Establecer 'datetime' como índice
        n_incoming_veh_df.set_index('datetime', inplace=True)

        # Renombrar la columna 'num_vehicles'
        n_incoming_veh_df.rename(columns={0: "num_vehicles"}, inplace=True)

        # Eliminar columnas originales si ya no se necesitan
        n_incoming_veh_df.drop(columns=["date"], inplace=True)

        self._scaler = MinMaxScaler()
        n_incoming_veh_df['num_vehicles'] = self._scaler.fit_transform(n_incoming_veh_df[['num_vehicles']])

        values = n_incoming_veh_df['num_vehicles'].values
        X, y = self._create_unidimensional_sequences(values, self._sequence_length)

        # Dividir los datos en conjunto de entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1-lr, random_state=42)

        # Cambiar la forma de los datos para adaptarse a la entrada CNN-LSTM
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

        self._last_sequence = X_test[-1].flatten() 

        if os.path.exists(model_path) and not refresh_model:
            print(f"\n\tLoading demand predictor from {model_path}...", end="")
            self._model = load_model(model_path)
        else: 
            print(f"\n\tTraining demand predictor...", end="")

            # Definir el modelo CNN-LSTM
            self._model = Sequential([
                Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)),
                MaxPooling1D(pool_size=2),
                LSTM(50, activation='relu', return_sequences=False),
                Dense(1)
            ])

            self._model.compile(optimizer='adam', loss='mse')

            _verbose= 0
            if show_details:
                _verbose= 1
            # Configuración de EarlyStopping
            early_stopping = EarlyStopping(
                monitor="val_loss",  # Métrica que se monitorea
                patience=10,         # Número de épocas de espera sin mejoras antes de detener
                restore_best_weights=True,  # Restaurar los mejores pesos
                verbose=_verbose          # Mostrar mensajes
            )

            # Entrenamiento con EarlyStopping
            self._model.fit(
                X_train,
                y_train,
                epochs=1000,
                batch_size=16,
                validation_data=(X_test, y_test),
                callbacks=[early_stopping],  # Incluir el callback
                verbose=1
            )
            
            self._model.save(model_path)

        print("DONE!")

    # Crear los datos de entrada y salida para la serie temporal
    def _create_unidimensional_sequences(self, data, sequence_length):
        X, y = [], []
        for i in range(len(data) - sequence_length):
            X.append(data[i:i + sequence_length])
            y.append(data[i + sequence_length])
        return np.array(X), np.array(y)
    
    def _predict_occupancy(self, n_days_ahead):

        future_predictions = []
        current_sequence = self._last_sequence.copy()

        for _ in range(n_days_ahead*24):
            # Redimensionar la secuencia actual para que sea compatible con el modelo
            input_data = np.array(current_sequence).reshape((1, self._sequence_length, 1))
            
            # Predecir el siguiente valor
            next_pred = self._model.predict(input_data, verbose=0)[0][0]
            
            # Guardar el valor predicho (desnormalizado)
            future_predictions.append(self._scaler.inverse_transform([[next_pred]])[0][0])

            # Actualizar la secuencia de entrada con la nueva predicción
            current_sequence = np.append(current_sequence[1:], next_pred)

        return future_predictions

    def _generate_entry_and_exit_hours(self, n_tuples):
        new_samples, _ = gm.sample(n_samples=n_tuples)
        return [[round(x[0]),round(x[1])] for x in new_samples]
    
    def generate_occupancy(self, num_days_ahead):
        """
        Simulates vehicle parking occupancy and generates a table of occupancy information.
        """

        n_vehicles_per_hour = self._predict_occupancy(num_days_ahead)
        simulated_occupancy=[]
        hour= 0
        for n_vehicles in n_vehicles_per_hour: 
            if hour in self._gmm_models:
                gm = self._gmm_models[hour]      
                new_samples, _ = gm.sample(n_samples=n_vehicles)
                simulated_occupancy.append([(hour, round(x[0])) for x in new_samples])
            hour = (hour+1) % 24
        return simulated_occupancy 

In [24]:
class AmbientModule:
    
    def __init__(self, lat, lon):
        self.lat= lat
        self.lon= lon

    def train_ambient_temperature_model(self, start_date, end_date, lr=0.9, show_details=False, refresh_model=False):
        """
        Trains a LSTM model using the combined temperature data.

        :param combined_temp_data: DataFrame containing combined temperature data
        """

        ambient_temperaure_df= self._fetch_historical_temperature(start_date, end_date)
        model_path = os.path.join('_models', 'cabintemp_lstm_model.keras')

        # Normalización de los datos
        scaler = MinMaxScaler(feature_range=(0, 1))
        #scaler_b = MinMaxScaler(feature_range=(0, 1))

        data = ambient_temperaure_df.copy()
        data["temperature_2m"] = scaler.fit_transform(ambient_temperaure_df["temperature_2m"].values.reshape(-1, 1))

        look_back = 12  # Número de pasos anteriores a considerar
        X, y = ClimaticPark.create_sequences(data.values, look_back)

        # Dividimos en entrenamiento y prueba
        train_size = int(len(X) * lr)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]

        # Redimensionamos las entradas para LSTM [samples, time steps, features]
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

        self._last_sequence = X_test[-1].flatten() 
        
        model = None
        # Cargamos o entrenamos el modelo
        if os.path.exists(model_path) and not refresh_model:
            print(f"\n\tLoading ambient temperature predictor from {model_path}...", end="")
            model = load_model(model_path)
            print("DONE!")
        else:
            print(f"\n\tTraining and saving model in {model_path}...")

            _verbose= 0
            if show_details:
              _verbose= 1
            # Configuración de EarlyStopping
            early_stopping = EarlyStopping(
                monitor="val_loss",  # Métrica que se monitorea
                patience=10,         # Número de épocas de espera sin mejoras antes de detener
                restore_best_weights=True,  # Restaurar los mejores pesos
                verbose=_verbose           # Mostrar mensajes
            )


            # Construcción del modelo LSTM
            model = Sequential()
            model.add(LSTM(50, input_shape=(look_back, 1)))
            model.add(Dense(1))
            model.compile(optimizer="adam", loss="mse")

            # Entrenamiento con EarlyStopping
            model.fit(
                X_train,
                y_train,
                epochs=1000,
                batch_size=16,
                validation_data=(X_test, y_test),
                callbacks=[early_stopping],  # Incluir el callback
                verbose=1
            )
            self._last_sequence = X_test[-1].flatten() 

        model.save(model_path)

        self._model= model
        self._scaler= scaler

    def predict_ambient_temperature(self, n_days_ahead):

        future_predictions = []
        current_sequence = self._last_sequence.copy()

        for _ in range(n_days_ahead*24):
            # Redimensionar la secuencia actual para que sea compatible con el modelo
            input_data = np.array(current_sequence).reshape((1, len(current_sequence), 1))
            
            # Predecir el siguiente valor
            next_pred = self._model.predict(input_data, verbose=0)[0][0]
            
            # Guardar el valor predicho (desnormalizado)
            future_predictions.append(self._scaler.inverse_transform([[next_pred]])[0][0])

            # Actualizar la secuencia de entrada con la nueva predicción
            current_sequence = np.append(current_sequence[1:], next_pred)

        return future_predictions

    # Función para obtener datos horarios históricos de Open-Meteo API
    def _fetch_historical_temperature(self, start_date, end_date):
          """
          Obtiene datos horarios históricos de Open-Meteo API.

          Args:
          - latitude (float): Latitud de la ubicación.
          - longitude (float): Longitud de la ubicación.
          - start_date (str): Fecha de inicio en formato YYYY-MM-DD.
          - end_date (str): Fecha de fin en formato YYYY-MM-DD.
          - parameters (list): Variables meteorológicas a consultar, ej. ['temperature_2m', 'humidity_2m'].

          Returns:
          - pd.DataFrame: Datos meteorológicos horarios como DataFrame.
          """
          base_url = "https://archive-api.open-meteo.com/v1/archive"


          # Crear el payload para la solicitud
          payload = {
              "latitude": self.lat,
              "longitude": self.lon,
              "start_date": start_date,
              "end_date": end_date,
              "hourly": 'temperature_2m',
              "timezone": "auto"
          }

          # Realizar la solicitud a la API
          response = requests.get(base_url, params=payload)

          if response.status_code == 200:
              # Convertir la respuesta JSON a un DataFrame
              data = response.json()
              if "hourly" in data:
                  df = pd.DataFrame(data["hourly"])
                  df['time'] = pd.to_datetime(df['time'])
                  df= df.set_index('time')
                  return df
              else:
                  print("No se encontraron datos en la respuesta.")
                  return pd.DataFrame()
          else:
              print(f"Error en la solicitud: {response.status_code} - {response.text}")
              return pd.DataFrame()
          

    def _forecast_uncovered_cabin_temperatures(self, ambient_temp):
        ambient_temp_scaled = self.temp_scaler.fit_transform(ambient_temp)
        y_pred = self.cabin_temp_model.predict(ambient_temp_scaled)
        y_pred_rescaled = self.cabin_temp_scaler.inverse_transform(y_pred)
        return y_pred_rescaled

In [25]:
class ClimaticPark:
    def __init__(self, file_name_lots='data/parking_lots.geojson',
                 file_name_roofs='data/parking_roofs.geojson',
                 file_name_coords='data/parking_coordinates.csv',
                 file_name_gates='data/gates_coordinates.csv',
                 file_name_mapcenter='data/map_center_coordinates.csv',
                 file_name_cabintem='data/historical_cabin_temp.csv',
                 file_name_tuples='data/entry_exit_tuples.csv'):
        """
        Initializes the ClimaticPark object by loading all necessary files.
        """
        # Load GeoJSON files for lots and roofs
        self.lots_data = ClimaticPark.load_geojson(file_name_lots)
        self.roofs_data = ClimaticPark.load_geojson(file_name_roofs)

        # Add 'height' column to roofs_data with a value of 1 for all rows
        if (self.lots_data is not None) and ('height' not in self.lots_data.columns):
            self.lots_data['height'] = 1  # Assuming a default height of 1

        # Add 'height' column to roofs_data with a value of 1 for all rows
        if (self.roofs_data is not None) and ('height' not in self.roofs_data.columns):
            self.roofs_data['height'] = 1  # Assuming a default height of 1

        # Load CSV files for coordinates, historical data, and additional data
        self.coords_data = ClimaticPark.load_csv(file_name_coords)
        self.gates_data = ClimaticPark.load_csv(file_name_gates)
      
        self.data_no_roof =  ClimaticPark.load_csv(os.path.join('data', 'cabin_temperature_no_roof.csv'))
        self.data_no_roof['coverage']=0
        self.data_roof =  ClimaticPark.load_csv(os.path.join('data', 'cabin_temperature_w_roof.csv'))
        self.data_roof['coverage']=1

        self.recorded_cabin_temp = pd.read_csv(file_name_cabintem, index_col=0)
        # Convertir la columna DateTime a tipo datetime
        self.recorded_cabin_temp['DateTime'] = pd.to_datetime(self.recorded_cabin_temp['DateTime'])
        # Establecer la columna DateTime como índice
        self.recorded_cabin_temp.set_index('DateTime', inplace=True)
        # Remuestrear los datos para obtener una frecuencia de 1 hora (calculando la media)
        self.recorded_cabin_temp = self.recorded_cabin_temp.resample('h').mean()

        self.entry_exit_tuples= pd.read_csv(os.path.join('data', 'entry_exit_tuples_clean.csv'), index_col=0, dtype={'id_subject':str}, parse_dates=['date'])
        
        os.makedirs('_models', exist_ok=True)
        self.cabin_temp_model = None
        self.cabin_coverage_model= None
        print("Generating Demand Module...", end="")
        self.demand_module = DemandModule(self.entry_exit_tuples)
        print("DONE!")
        
        lat = self.coords_data['latitude'].iloc[0]  
        lon = self.coords_data['longitude'].iloc[0]
        
        print("Generating Shadow Module...", end="")
        self.shadow_module = ShadowModule(lat, lon, self.roofs_data, self.lots_data)
        print("DONE!")

        print("Generating Ambient Module...", end="")
        #self.ambient_module = AmbientModule(lat,lon)
        print("DONE!")

    @staticmethod
    def load_geojson(file_name):
        """
        Loads a GeoJSON file into a GeoDataFrame.
        """
        if file_name:
            return gpd.read_file(file_name).set_geometry("geometry")
        else:
            print(f"No GeoJSON file provided for {file_name}.")
            return None

    @staticmethod
    def load_csv(file_name):
        """
        Loads a CSV file into a DataFrame.
        """
        if file_name:
            return pd.read_csv(file_name)
        else:
            print(f"No CSV file provided for {file_name}.")
            return None

    def prepare_simulation(self, lr=0.8, display_details=False):
        print("Preparing simulation for TPL...")
        """
        if not self.coords_data.empty:
            latitude = self.coords_data['latitude'].iloc[0]  # Get the first coordinate
            longitude = self.coords_data['longitude'].iloc[0]
        else:
            raise ValueError("The coordinates file is empty or not formatted correctly.")
        """
        init_date =  self.recorded_cabin_temp.index[0].date()
        final_date =  self.recorded_cabin_temp.index[-1].date()

        # Process temperature data
        print("\tTraining ambient temperature predictors...",end="")
        #self.ambient_module.train_ambient_temperature_model(init_date, final_date)
        #combined_temp_df= pd.concat([ambient_temp_df, self.recorded_cabin_temp], axis=1)
        #combined_temp_df = combined_temp_df.dropna()


        #print("Training cabin temperature predictors...",end="")
        #self.cabin_temp_model, self.temp_scaler, self.cabin_temp_scaler = self._train_cabin_temperature_model(combined_temp_df, lr, display_details)
        #self.cabin_coverage_model= self._train_cabin_temperature_and_coverage_model()
        #print("DONE!")

        print("\tTraining demand predictors...",end="")
        self.demand_module.train_demand_predictors()  

        print("Simulation ready to go!!")

    def launch_simulation(self, n_days_ahead, display_details=True):
        """
        Lauch simulation for n_days_ahead 
        """
        print("Starting simulation of TPL...")

        """
        if not self.coords_data.empty:
            latitude = self.coords_data['latitude'].iloc[0]  # Get the first coordinate
            longitude = self.coords_data['longitude'].iloc[0]
        else:
            raise ValueError("The coordinates file is empty or not formatted correctly.")
        """

        self.demand_module.generate_occupancy(n_days_ahead)
        
        init_day = self.entry_exit_tuples['date'].max().date()

        date_lst = [init_day + timedelta(days=i) for i in range(n_days_ahead)]
        print(date_lst)


        #simulated_ambient_temp= self.ambient_module.predict_ambient_temperature(n_days_ahead) 
        #print(len(simulated_ambient_temp), simulated_ambient_temp)


        simulated_occupancy = self.demand_module.generate_occupancy(n_days_ahead)    
        #print(len(simulated_occupancy), simulated_occupancy)    

        simulated_shadows= self.shadow_module.compute_coverage_rates(date_lst)
        print(simulated_shadows)

    # Preparamos el dataset para secuencias
    @staticmethod
    def create_sequences(data, look_back):
        X, y = [], []
        for i in range(len(data) - look_back):
            X.append(data[i:i + look_back])  # Secuencias de la variable 'a'
            y.append(data[i + look_back])  # Predicción futura de la variable 'b'
        return np.array(X), np.array(y)

    def _train_cabin_temperature_and_coverage_model(self):

        temp_data = pd.concat([self.data_no_roof, self.data_roof], axis=0)

        X= temp_data['T temp_ext coverage'.split()].values
        y=  temp_data['temp_int'].values

        clf = LinearRegression().fit(X, y)
        return clf

In [26]:
park = ClimaticPark() # default parameters
park.prepare_simulation()

Generating Demand Module...DONE!
Generating Shadow Module...DONE!
Generating Ambient Module...DONE!
Preparing simulation for TPL...
	Training ambient temperature predictors...	Training demand predictors...
	Loading demand predictor from _models\demand_cnnlstm_model.keras...DONE!
Simulation ready to go!!


In [27]:
park.launch_simulation(4)

Starting simulation of TPL...
[datetime.date(2022, 10, 27), datetime.date(2022, 10, 28), datetime.date(2022, 10, 29), datetime.date(2022, 10, 30)]
                                              geometry  \
0    POLYGON ((-1.18571 37.99249, -1.18573 37.99248...   
1    POLYGON ((-1.18567 37.99250, -1.18570 37.99249...   
2    POLYGON ((-1.18564 37.99251, -1.18567 37.99250...   
3    POLYGON ((-1.18561 37.99252, -1.18564 37.99251...   
4    POLYGON ((-1.18558 37.99253, -1.18561 37.99252...   
..                                                 ...   
204  POLYGON ((-1.18530 37.99281, -1.18532 37.99280...   
205  POLYGON ((-1.18526 37.99282, -1.18529 37.99281...   
206  POLYGON ((-1.18523 37.99283, -1.18526 37.99282...   
207  POLYGON ((-1.18520 37.99284, -1.18523 37.99283...   
208  POLYGON ((-1.18517 37.99285, -1.18520 37.99284...   

     coverage_rate_2022-10-27 06:00  coverage_rate_2022-10-27 07:00  \
0                          0.538525                        0.538605   
1             

In [28]:
"""
raw_park_records_df= pd.read_csv(os.path.join('data', 'entry_exit_tuples_clean.csv'), index_col=0,
                              dtype={'id_subject':str}, parse_dates=['date'])
raw_park_records_df.head()
"""

"\nraw_park_records_df= pd.read_csv(os.path.join('data', 'entry_exit_tuples_clean.csv'), index_col=0,\n                              dtype={'id_subject':str}, parse_dates=['date'])\nraw_park_records_df.head()\n"

In [29]:
"""
init_day= raw_park_records_df['date'].max()
print(init_day)
n_days = 4
date_lst = [init_day + timedelta(days=i) for i in range(n_days)]
days_lst= [date.date() for date in date_lst]

print(days_lst)
"""

"\ninit_day= raw_park_records_df['date'].max()\nprint(init_day)\nn_days = 4\ndate_lst = [init_day + timedelta(days=i) for i in range(n_days)]\ndays_lst= [date.date() for date in date_lst]\n\nprint(days_lst)\n"

In [30]:
"""
n_components = 2  # Número de componentes de la mezcla
gm = GaussianMixture(n_components=n_components, random_state=42)
gm.fit(raw_park_records_df[['enter_hour', 'exit_hour']])

new_samples, _ = gm.sample(n_samples=100)
[[round(x[0]),round(x[1])] for x in new_samples]
"""

"\nn_components = 2  # Número de componentes de la mezcla\ngm = GaussianMixture(n_components=n_components, random_state=42)\ngm.fit(raw_park_records_df[['enter_hour', 'exit_hour']])\n\nnew_samples, _ = gm.sample(n_samples=100)\n[[round(x[0]),round(x[1])] for x in new_samples]\n"

In [31]:
"""
n_incoming_veh_df= raw_park_records_df.groupby('date').size().reset_index()
# Crear columna datetime a partir de 'date' y 'enter_hour'
n_incoming_veh_df['datetime'] = pd.to_datetime(n_incoming_veh_df['date'])

# Establecer 'datetime' como índice
n_incoming_veh_df.set_index('datetime', inplace=True)

# Renombrar la columna 'num_vehicles'
n_incoming_veh_df.rename(columns={0: "num_vehicles"}, inplace=True)

# Eliminar columnas originales si ya no se necesitan
n_incoming_veh_df.drop(columns=["date"], inplace=True)
n_incoming_veh_df
"""

'\nn_incoming_veh_df= raw_park_records_df.groupby(\'date\').size().reset_index()\n# Crear columna datetime a partir de \'date\' y \'enter_hour\'\nn_incoming_veh_df[\'datetime\'] = pd.to_datetime(n_incoming_veh_df[\'date\'])\n\n# Establecer \'datetime\' como índice\nn_incoming_veh_df.set_index(\'datetime\', inplace=True)\n\n# Renombrar la columna \'num_vehicles\'\nn_incoming_veh_df.rename(columns={0: "num_vehicles"}, inplace=True)\n\n# Eliminar columnas originales si ya no se necesitan\nn_incoming_veh_df.drop(columns=["date"], inplace=True)\nn_incoming_veh_df\n'

In [32]:
"""
ax=n_incoming_veh_df.plot(figsize=(15,5), grid=True)
ax.set_xlabel('Date',fontsize=20);
ax.set_ylabel('Num. of hourly users',fontsize=20);
ax.tick_params(axis='y', labelsize=15)
ax.tick_params(axis='x', labelsize=15)
"""

"\nax=n_incoming_veh_df.plot(figsize=(15,5), grid=True)\nax.set_xlabel('Date',fontsize=20);\nax.set_ylabel('Num. of hourly users',fontsize=20);\nax.tick_params(axis='y', labelsize=15)\nax.tick_params(axis='x', labelsize=15)\n"

In [33]:
"""
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

scaler = MinMaxScaler()
n_incoming_veh_df['num_vehicles'] = scaler.fit_transform(n_incoming_veh_df[['num_vehicles']])

# Crear los datos de entrada y salida para la serie temporal
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length])
        y.append(data[i + sequence_length])
    return np.array(X), np.array(y)

sequence_length = 12  # Longitud de las secuencias
values = n_incoming_veh_df['num_vehicles'].values
X, y = create_sequences(values, sequence_length)

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Cambiar la forma de los datos para adaptarse a la entrada CNN-LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Definir el modelo CNN-LSTM
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.LSTM(50, activation='relu', return_sequences=False),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Configuración de EarlyStopping
early_stopping = EarlyStopping(
    monitor="val_loss",  # Métrica que se monitorea
    patience=10,         # Número de épocas de espera sin mejoras antes de detener
    restore_best_weights=True,  # Restaurar los mejores pesos
    verbose=1          # Mostrar mensajes
)

# Entrenar el modelo
#history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# Entrenamiento con EarlyStopping
model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=16,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],  # Incluir el callback
    verbose=1
)


# Evaluar el modelo
loss = model.evaluate(X_test, y_test)
print(f"Pérdida en el conjunto de prueba: {loss}")

# Hacer predicciones
y_pred = model.predict(X_test)

# Desescalar los resultados para obtener valores originales
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_rescaled = scaler.inverse_transform(y_pred)

print(f"Valores originales predichos: {y_pred_rescaled[:5].flatten()}")

"""

'\nimport numpy as np\nimport pandas as pd\nimport tensorflow as tf\nfrom sklearn.preprocessing import MinMaxScaler\nfrom sklearn.model_selection import train_test_split\n\nscaler = MinMaxScaler()\nn_incoming_veh_df[\'num_vehicles\'] = scaler.fit_transform(n_incoming_veh_df[[\'num_vehicles\']])\n\n# Crear los datos de entrada y salida para la serie temporal\ndef create_sequences(data, sequence_length):\n    X, y = [], []\n    for i in range(len(data) - sequence_length):\n        X.append(data[i:i + sequence_length])\n        y.append(data[i + sequence_length])\n    return np.array(X), np.array(y)\n\nsequence_length = 12  # Longitud de las secuencias\nvalues = n_incoming_veh_df[\'num_vehicles\'].values\nX, y = create_sequences(values, sequence_length)\n\n# Dividir los datos en conjunto de entrenamiento y prueba\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# Cambiar la forma de los datos para adaptarse a la entrada CNN-LSTM\nX_train = X_tr

In [34]:
"""
import requests
import pandas as pd

# Función para obtener datos horarios históricos de Open-Meteo API
def get_historical_weather_hourly(latitude, longitude, start_date, end_date):
    """
    Obtiene datos horarios históricos de Open-Meteo API.

    Args:
    - latitude (float): Latitud de la ubicación.
    - longitude (float): Longitud de la ubicación.
    - start_date (str): Fecha de inicio en formato YYYY-MM-DD.
    - end_date (str): Fecha de fin en formato YYYY-MM-DD.
    - parameters (list): Variables meteorológicas a consultar, ej. ['temperature_2m', 'humidity_2m'].

    Returns:
    - pd.DataFrame: Datos meteorológicos horarios como DataFrame.
    """
    base_url = "https://archive-api.open-meteo.com/v1/archive"


    # Crear el payload para la solicitud
    payload = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": 'temperature_2m',
        "timezone": "auto"
    }

    # Realizar la solicitud a la API
    response = requests.get(base_url, params=payload)

    if response.status_code == 200:
        # Convertir la respuesta JSON a un DataFrame
        data = response.json()
        if "hourly" in data:
            df = pd.DataFrame(data["hourly"])

            return df
        else:
            print("No se encontraron datos en la respuesta.")
            return pd.DataFrame()
    else:
        print(f"Error en la solicitud: {response.status_code} - {response.text}")
        return pd.DataFrame()

ambient_temperatures_df = get_historical_weather_hourly(park.coords_data['latitude'].iloc[0], park.coords_data['longitude'].iloc[0], init_date, final_date)
ambient_temperatures_df
"""

IndentationError: unexpected indent (3025937763.py, line 8)